In [28]:
import plotly.graph_objects as go
from datetime import date
from numpy import array, zeros, linalg, pi, arange, sin, pi
from math import atan2

from func.matrix_1_4 import matrix_1_4
from func.deg_to_rad import deg_to_rad
from func.rotation_matrix import rotation_matrix
from func.transition_matrix import transition_matrix
from func.cross_normal_to_oz import cross_normal_to_oz
from func.added_first_number import a_f_n
from arms_degrees_with_angular_limit_on_joint_and_arm import solve
from func.zero_matrix import zero_matrix

### Объявим размеры используемых матриц:

In [2]:
matrix_move_lower_plate = zero_matrix(1) # Нижняя плита
lower_points = zero_matrix(2) 

matrix_move_upper_plate = zero_matrix(1) # Верхняя плита
upper_points = zero_matrix(2)

position_upper_from_lower = array([zeros((4,1)), zeros((4,1)), zeros((4,1)), zeros((4,1)), zeros((4,1)), zeros((4,1))]) # Пересечение
coordinates_cross_matrix = array([zeros((4,1)), zeros((4,1)), zeros((4,1)), zeros((4,1)), zeros((4,1)), zeros((4,1))]) 
global_cross_coordinates = array([zeros((4,1)), zeros((4,1)), zeros((4,1)), zeros((4,1)), zeros((4,1)), zeros((4,1))]) 
degrees_cross_matrix = zeros(6) 

coordinate_system_lower_plate = array([zeros((4,1)), zeros((4,1)), zeros((4,1)), zeros((4,1))]) # Для Системы координат
coordinate_system_upper_plate = array([zeros((4,1)), zeros((4,1)), zeros((4,1)), zeros((4,1))])
coordinate_system_lower_point = array([[array([zeros((4,1)), zeros((4,1)), zeros((4,1)), zeros((4,1))])], # Для каждой нижней точки записываю локальную СК
                                    [array([zeros((4,1)), zeros((4,1)), zeros((4,1)), zeros((4,1))])],
                                    [array([zeros((4,1)), zeros((4,1)), zeros((4,1)), zeros((4,1))])],
                                    [array([zeros((4,1)), zeros((4,1)), zeros((4,1)), zeros((4,1))])],
                                    [array([zeros((4,1)), zeros((4,1)), zeros((4,1)), zeros((4,1))])],
                                    [array([zeros((4,1)), zeros((4,1)), zeros((4,1)), zeros((4,1))])],])
coordinate_system_upper_point = array([[array([zeros((4,1)), zeros((4,1)), zeros((4,1)), zeros((4,1))])],
                                    [array([zeros((4,1)), zeros((4,1)), zeros((4,1)), zeros((4,1))])],
                                    [array([zeros((4,1)), zeros((4,1)), zeros((4,1)), zeros((4,1))])],
                                    [array([zeros((4,1)), zeros((4,1)), zeros((4,1)), zeros((4,1))])],
                                    [array([zeros((4,1)), zeros((4,1)), zeros((4,1)), zeros((4,1))])],
                                    [array([zeros((4,1)), zeros((4,1)), zeros((4,1)), zeros((4,1))])],])                                    
matrix_move_crosspoints_cs = array([zeros((4,4)), zeros((4,4)), zeros((4,4)), zeros((4,4)), zeros((4,4)), zeros((4,4))])
coordinate_system_crosspoints = array([[array([zeros((4,1)), zeros((4,1)), zeros((4,1)), zeros((4,1))])],
                                    [array([zeros((4,1)), zeros((4,1)), zeros((4,1)), zeros((4,1))])],
                                    [array([zeros((4,1)), zeros((4,1)), zeros((4,1)), zeros((4,1))])],
                                    [array([zeros((4,1)), zeros((4,1)), zeros((4,1)), zeros((4,1))])],
                                    [array([zeros((4,1)), zeros((4,1)), zeros((4,1)), zeros((4,1))])],
                                    [array([zeros((4,1)), zeros((4,1)), zeros((4,1)), zeros((4,1))])]])
# Коорд. верхней точки относительно пересечения
coordinates_upperPoints_from_crosspoints = array([zeros((4,1)), zeros((4,1)), zeros((4,1)), zeros((4,1)), zeros((4,1)), zeros((4,1))])  
angle_upper_points_from_crosspoints_on_ZX = zeros(6)
angle_upper_points_from_crosspoints_on_XY = zeros(6)
# Поворот СК в (.) пересечения
matrix_rotate_angle_upperPoints_from_crosspoints_on_ZX = array([zeros((4,4)), zeros((4,4)), zeros((4,4)), zeros((4,4)), zeros((4,4)), zeros((4,4))])    
matrix_move_crosspoints_cs_2 = array([zeros((4,4)), zeros((4,4)), zeros((4,4)), zeros((4,4)), zeros((4,4)), zeros((4,4))]) 
coordinates_upperPoints_from_crosspoints_in_new_planes = array([zeros((4,1)), zeros((4,1)), zeros((4,1)), zeros((4,1)), zeros((4,1)), zeros((4,1))])
angle_joint_lower = zeros(6) 
coordinate_system_crosspoints_after_rotate = array([[array([zeros((4,1)), zeros((4,1)), zeros((4,1)), zeros((4,1))])],
                                    [array([zeros((4,1)), zeros((4,1)), zeros((4,1)), zeros((4,1))])],
                                    [array([zeros((4,1)), zeros((4,1)), zeros((4,1)), zeros((4,1))])],
                                    [array([zeros((4,1)), zeros((4,1)), zeros((4,1)), zeros((4,1))])],
                                    [array([zeros((4,1)), zeros((4,1)), zeros((4,1)), zeros((4,1))])],
                                    [array([zeros((4,1)), zeros((4,1)), zeros((4,1)), zeros((4,1))])]])
coordinates_crosspoints_from_lower_points = array([zeros((4,1)), zeros((4,1)), zeros((4,1)), zeros((4,1)), zeros((4,1)), zeros((4,1))])
angle_crosspoints_from_lower_points_on_XY = zeros(6)
matrix_rotate_angle_crosspoints_from_lower_points_on_XZ = array([zeros((4,4)), zeros((4,4)), zeros((4,4)), zeros((4,4)), zeros((4,4)), zeros((4,4))])
matrix_move_crosspoints_cs_3 = array([zeros((4,4)), zeros((4,4)), zeros((4,4)), zeros((4,4)), zeros((4,4)), zeros((4,4))])
coordinates_crosspoints_from_lower_points_new_planes = array([zeros((4,1)), zeros((4,1)), zeros((4,1)), zeros((4,1)), zeros((4,1)), zeros((4,1))])
angle_joint_upper = zeros(6) 

# Расчёт нижнего шарнира пружины
coordinates_upper_joint_of_gas_spring_from_lower = array([zeros((4,1)), zeros((4,1)), zeros((4,1))])
angles_between_spring_and_ZY_of_lower_joint = zeros(3) 
matrix_rotate_cs_lower_joint_of_gas_spring =  array([zeros((4,4)), zeros((4,4)), zeros((4,4))])
matrix_move_to_new_cs_of_lower_joint_of_gas_spring = array([zeros((4,4)), zeros((4,4)), zeros((4,4))])
coordinates_upper_joint_gas_spring_from_lower_in_new_planes = array([zeros((4,1)), zeros((4,1)), zeros((4,1))])
angle_lower_joint_of_gas_spring = zeros(3) 
coordinate_system_cs_lower_joint_of_gas_spring = array([[array([zeros((4,1)), zeros((4,1)), zeros((4,1)), zeros((4,1))])],
                                                    [array([zeros((4,1)), zeros((4,1)), zeros((4,1)), zeros((4,1))])],
                                                    [array([zeros((4,1)), zeros((4,1)), zeros((4,1)), zeros((4,1))])]])
# Расчёт верхнего шарнира пружины
coordinates_lower_joint_of_gas_spring_from_upper = array([zeros((4,1)), zeros((4,1)), zeros((4,1))])
angles_between_spring_and_ZY_of_upper_joint = zeros(3) 
matrix_rotate_cs_upper_joint_of_gas_spring = array([zeros((4,4)), zeros((4,4)), zeros((4,4))])
matrix_move_to_new_cs_of_upper_joint_of_gas_spring = array([zeros((4,4)), zeros((4,4)), zeros((4,4))])
coordinates_lower_joint_gas_spring_from_upper_in_new_planes = array([zeros((4,1)), zeros((4,1)), zeros((4,1))])
angle_upper_joint_of_gas_spring = zeros(3) 
coordinate_system_cs_upper_joint_of_gas_spring = array([[array([zeros((4,1)), zeros((4,1)), zeros((4,1)), zeros((4,1))])],
                                                    [array([zeros((4,1)), zeros((4,1)), zeros((4,1)), zeros((4,1))])],
                                                    [array([zeros((4,1)), zeros((4,1)), zeros((4,1)), zeros((4,1))])]])

# 1. Координаты плит

## Нижняя плита

In [3]:
"""Смещения нижней плиты"""

offset_lower = array([[-307, -125 , 0, 0, 0, 0], # Угол gamma подбирался так, чтобы OY лежащих на одной прямой точек были направлены друг на друга
                    [45.25,  -328.37, 0, 0, 0, -60],
                    [261.75,  -203.37, 0, 0, 0, 120], 
                    [261.75,  203.37, 0, 0, 0, 60],   # X, Y, Z, alpha, beta, gamma
                    [45.25,  328.37, 0, 0, 0, -120], 
                    [-307,  125, 0, 0, 0, 180],
                    [-359, 0, -292, 0, 0, 0], # Координаты крепления пружин на нижней плите
                    [179.5, -310.9, -292, 0, 0, 120],
                    [179.5, 310.9, -292, 0, 0, -120]])  

# offset_lower = array([[125, -307 , 0, 0, 0, 0], 
#                     [328.37, 45.25, 0, 0, 0, -60],
#                     [203.37, 261.75, 0, 0, 0, 120], 
#                     [-203.37, 261.75, 0, 0, 0, 60],   
#                     [-328.37, 45.25, 0, 0, 0, -120], 
#                     [-125, -307, 0, 0, 0, 180],
#                     [0, -359, -292, 0, 0, 0], 
#                     [310.9, 179.5, -292, 0, 0, 120],
#                     [-310.9, 179.5, -292, 0, 0, -120]]) 

rotate_lower_local_cs = array([0, 0, 0, 0, 90, 90]) # Поворачиваю нижние локальные СК так, чтобы ось вращения стала OZ

rotate_main_cs = array([0, 0, 0, 0, 0, 0])    # Поворот главной нижней оси

In [4]:
"""Запишем матрицы перемещений для каждого центра тяги в один массив"""

matrix_rotate_lower_local_cs = rotation_matrix(deg_to_rad(rotate_lower_local_cs[3]), deg_to_rad(rotate_lower_local_cs[4]), deg_to_rad(rotate_lower_local_cs[5]))

matrix_rotate_main_cs = rotation_matrix(deg_to_rad(rotate_main_cs[3]), deg_to_rad(rotate_main_cs[4]), deg_to_rad(rotate_main_cs[5]))

for i in range(len(offset_lower)):
    matrix_move_lower_plate[i] = matrix_rotate_main_cs.dot((transition_matrix(offset_lower[i][0], offset_lower[i][1], offset_lower[i][2]).dot(
                                rotation_matrix(deg_to_rad(offset_lower[i][3]), deg_to_rad(offset_lower[i][4]), deg_to_rad(offset_lower[i][5])))).dot(matrix_rotate_lower_local_cs))

In [5]:
local_lower_CS = matrix_1_4(0, 0, 0)   # Пусть центр тяги находится в (0, 0, 0) в локальных координатах

In [6]:
"""Координаты нижней плиты находим через перемножение матрицы перемещения на локальную СК"""

for i in range(len(offset_lower)):
    lower_points[i] = matrix_move_lower_plate[i].dot(local_lower_CS)

## Верхняя плита

In [7]:
move_upper_CS = array([0, 0, 300, 0, 0, 0])   # Смещение центра верхней плиты    25, 25, 300, 5, 5, 5

# Координаты отсутпа верхней плиты (в нулевом положении)
offset_upper = array([[-307,  -60, 0, 0, 0, 0],  # X, Y, Z, alpha, beta, gamma 
                    [101.54,  -295.87, 0, 0, 0, -60],
                    [205.46,  -235.87, 0, 0, 0, 120], 
                    [205.46,  235.87, 0, 0, 0, 60], 
                    [101.54,  295.87, 0, 0, 0, -120],
                    [-307,  60, 0, 0, 0, -180],
                    [-307, 0, 0, 0, 0, 0], # Координаты крепления пружин на верхней плите
                    [153.5, -265.87, 0, 0, 0, 120],
                    [153.5, 265.87, 0, 0, 0, -120]])

# offset_upper = array([[60, -307, 0, 0, 0, 0],
#                     [295.87, 101.54, 0, 0, 0, -60],
#                     [235.87, 205.46, 0, 0, 0, 120], 
#                     [-235.87, 205.46, 0, 0, 0, 60], 
#                     [-295.87, 101.54, 0, 0, 0, -120],
#                     [-60, -307, 0, 0, 0, -180],
#                     [0, -307, 0, 0, 0, 0],
#                     [265.87, 153.57, 0, 0, 0, 120],
#                     [-265.87, 153.5, 0, 0, 0, -120]])

In [8]:
local_upper_CS = matrix_1_4(0, 0, 0)

In [9]:
"""Перемещение и поворот верхней плиты"""
# Сначала смещаем верхнюю центральную точку, затем поворачиваем
matrix_move_upper_CS = matrix_rotate_main_cs.dot(transition_matrix(move_upper_CS[0], move_upper_CS[1], move_upper_CS[2]).dot(
                     rotation_matrix(deg_to_rad(move_upper_CS[3]), deg_to_rad(move_upper_CS[4]), deg_to_rad(move_upper_CS[5])))) 
                        
# Умножаю матрицу смещения верхней точки на матрицу смещения точек плиты
for i in range(len(offset_upper)):
   matrix_move_upper_plate[i] = matrix_move_upper_CS.dot((transition_matrix(offset_upper[i][0], offset_upper[i][1], offset_upper[i][2]).dot(    
                                rotation_matrix(deg_to_rad(offset_upper[i][3]), deg_to_rad(offset_upper[i][4]), deg_to_rad(offset_upper[i][5])))).dot(
                                matrix_rotate_lower_local_cs))

In [10]:
local_upper_CS_after_move = matrix_move_upper_CS.dot(local_upper_CS)   

for i in range(len(offset_upper)):
    upper_points[i] = matrix_move_upper_plate[i].dot(local_upper_CS) # Перемещение и поворот верхней плиты 

# 2. Точки пересечения

In [11]:
for i in range(6):
    position_upper_from_lower[i] = linalg.inv(matrix_move_lower_plate[i]).dot(upper_points[i])

In [12]:
R = 130   # Радиус окружности (рычага)
r = 300   # Радиус сферы

Результат пересечения в локальных координатах с отдельной матрицей для углов:

In [13]:
for i in range(len(coordinates_cross_matrix)):
    cross_matrix = cross_normal_to_oz(position_upper_from_lower[i], R, r)
    coordinates_cross_matrix[i] = matrix_1_4(cross_matrix[0], cross_matrix[1], cross_matrix[2])

    degrees_cross_matrix[i] = cross_normal_to_oz(position_upper_from_lower[i], R, r)[3]

In [14]:
coordinates_cross_matrix[0]

array([[129.80687143],
       [  7.08351119],
       [  0.        ],
       [  1.        ]])

Получив координаты пересечения тяги и рычага в локальной системе координат переведём их в глобальную:

In [15]:
for i in range(len(global_cross_coordinates)):
    global_cross_coordinates[i] =  matrix_move_lower_plate[i].dot(coordinates_cross_matrix[i])   # Эти локальные координаты являются частью объекта, который мы и переносим

# 3. Угол шаровой (и системы координат)

### Отрисуем системы координат на плитах

In [16]:
global_coordinate_system = array([[80, 0, 0],  # OX
                            [0, 80, 0], # OY
                            [0, 0, 80],# OZ 
                            [0, 0, 0]]) # 0, 0                         

In [17]:
for j in range(6):
    for i in range(len(global_coordinate_system)): 
        coordinate_system_upper_plate[i] = (matrix_move_upper_CS.dot(matrix_1_4(global_coordinate_system[i][0], 
                                            global_coordinate_system[i][1], global_coordinate_system[i][2])))   # СК центра верхней плиты
        coordinate_system_lower_point[j][0][i] = matrix_move_lower_plate[j].dot(matrix_1_4(global_coordinate_system[i][0], 
                                                                global_coordinate_system[i][1], global_coordinate_system[i][2]))   # Локальные СК нижней плиты                                                          
        # Матрица перемещения для СК точек пересечения
        matrix_move_crosspoints_cs[j] = matrix_move_lower_plate[j].dot(transition_matrix(coordinates_cross_matrix[j][0][0], coordinates_cross_matrix[j][1][0], coordinates_cross_matrix[j][2][0]))
        coordinate_system_crosspoints[j][0][i] = matrix_move_crosspoints_cs[j].dot(matrix_1_4(global_coordinate_system[i][0],   # Локальные СК точек пересечения
                                                                global_coordinate_system[i][1], global_coordinate_system[i][2]))

### Поиск угла нижних шарниров на рычагах

In [18]:
for i in range(6):
    # Находим координаты верхней шаровой относительно нижней (конца рычага)
    coordinates_upperPoints_from_crosspoints[i] = linalg.inv(matrix_move_crosspoints_cs[i]).dot(upper_points[i])
    # Найдём угол между проекцией тяги (на XY) и ZX, чтобы повернуть XY на полученный угол 
    # (тяга няходится в 1-ей четверти)
    angle_upper_points_from_crosspoints_on_XY[i] = atan2(coordinates_upperPoints_from_crosspoints[i][1][0], coordinates_upperPoints_from_crosspoints[i][0][0])
    matrix_rotate_angle_upperPoints_from_crosspoints_on_ZX[i] = rotation_matrix(0, 0, angle_upper_points_from_crosspoints_on_XY[i])
    # Создадим матрицу перемещения до точки пересечения с последующим поворотом до XZ
    matrix_move_crosspoints_cs_2[i] = matrix_move_crosspoints_cs[i].dot(matrix_rotate_angle_upperPoints_from_crosspoints_on_ZX[i])  
    # Узнаем координаты верхнего шарнира относительно нижнего в новой СК
    coordinates_upperPoints_from_crosspoints_in_new_planes[i] = linalg.inv(matrix_move_crosspoints_cs_2[i]).dot(upper_points[i])
    # Найдём углы
    angle_joint_lower[i] = atan2(coordinates_upperPoints_from_crosspoints_in_new_planes[i][2][0], coordinates_upperPoints_from_crosspoints_in_new_planes[i][0][0])

for i in range(6):
    for j in range(len(global_coordinate_system)): 
        coordinate_system_crosspoints_after_rotate[i][0][j] = matrix_move_crosspoints_cs_2[i].dot(matrix_1_4(global_coordinate_system[j][0],   # Локальные СК точек пересечения
                                                            global_coordinate_system[j][1], global_coordinate_system[j][2]))    

### Поиск угла верхнего шарнира

In [19]:
for i in range(6):
    # Находим координаты нижней шаровой относительно верхней
    coordinates_crosspoints_from_lower_points[i] = linalg.inv(matrix_move_upper_plate[i]).dot(global_cross_coordinates[i])
    # Найдём угол между тягой и XY, чтобы повернуть YZ на полученный угол (хотим, чтобы тяга лежала в плоскости ZY,
    # (тяга няходится в 3-ей четверти, поэтому проще повернуть плоскость YZ, а не XZ - так ближе)
    angle_crosspoints_from_lower_points_on_XY[i] = atan2(coordinates_crosspoints_from_lower_points[i][0][0], coordinates_crosspoints_from_lower_points[i][1][0])
    matrix_rotate_angle_crosspoints_from_lower_points_on_XZ[i] = rotation_matrix(0, 0, -angle_crosspoints_from_lower_points_on_XY[i])
    # Создадим матрицу перемещения до точки пересечения с последующим поворотом до XZ
    matrix_move_crosspoints_cs_3[i] = matrix_move_upper_plate[i].dot(matrix_rotate_angle_crosspoints_from_lower_points_on_XZ[i])  
    # Узнаем координаты нижнего шарнира относительно верхнего в новой СК
    coordinates_crosspoints_from_lower_points_new_planes[i] = linalg.inv(matrix_move_crosspoints_cs_3[i]).dot(global_cross_coordinates[i])
    # Найдём углы
    angle_joint_upper[i] = atan2(coordinates_crosspoints_from_lower_points_new_planes[i][2][0], coordinates_crosspoints_from_lower_points_new_planes[i][1][0])

for i in range(6):
    for j in range(len(global_coordinate_system)): 
        coordinate_system_upper_point[i][0][j] = matrix_move_crosspoints_cs_3[i].dot(matrix_1_4(global_coordinate_system[j][0], 
                                                                global_coordinate_system[j][1], global_coordinate_system[j][2]))    # Локальные СК верхней плиты    

### Поиск углов шарниров пружин

In [20]:
# Находим координаты верхней шаровой относительно нижней
for i in range(3):
    coordinates_upper_joint_of_gas_spring_from_lower[i] = linalg.inv(matrix_move_lower_plate[i+6]).dot(upper_points[i+6])
    # Найдём угол между проекцией пружины (на YX) и ZY, чтобы повернуть XY на полученный угол по OZ
    # (пружина няходится в 1-ей четверти)
    angles_between_spring_and_ZY_of_lower_joint[i] = atan2(coordinates_upper_joint_of_gas_spring_from_lower[i][0][0], coordinates_upper_joint_of_gas_spring_from_lower[i][1][0])
    # Матрицы поворота СК (чтобы пружина лежала в плоскости ZY)
    matrix_rotate_cs_lower_joint_of_gas_spring[i] = rotation_matrix(0, 0, -angles_between_spring_and_ZY_of_lower_joint[i])
    # Создадим общую матрицу перемещения с последующим поворотом
    matrix_move_to_new_cs_of_lower_joint_of_gas_spring[i] = matrix_move_lower_plate[i+6].dot(matrix_rotate_cs_lower_joint_of_gas_spring[i])
    # Узнаем координаты верхнего шарнира точки пересечения в новой СК
    coordinates_upper_joint_gas_spring_from_lower_in_new_planes[i] = linalg.inv(matrix_move_to_new_cs_of_lower_joint_of_gas_spring[i]).dot(upper_points[i+6])
    # # Найдём углы
    angle_lower_joint_of_gas_spring[i] = atan2(coordinates_upper_joint_gas_spring_from_lower_in_new_planes[i][2][0], coordinates_upper_joint_gas_spring_from_lower_in_new_planes[i][1][0])

for i in range(3):
    for j in range(len(global_coordinate_system)): 
        coordinate_system_cs_lower_joint_of_gas_spring[i][0][j] = matrix_move_to_new_cs_of_lower_joint_of_gas_spring[i].dot(matrix_1_4(global_coordinate_system[j][0],
                                                            global_coordinate_system[j][1], global_coordinate_system[j][2]))        
print(angle_lower_joint_of_gas_spring[0]*180/pi)

5.019853642019552


In [21]:
# Находим координаты нижней шаровой относительно верхней
for i in range(3):
  coordinates_lower_joint_of_gas_spring_from_upper[i] = linalg.inv(matrix_move_upper_plate[i+6]).dot(lower_points[i+6])
      # (пружина няходится в 3-ей четверти)
  angles_between_spring_and_ZY_of_upper_joint[i] = atan2(coordinates_lower_joint_of_gas_spring_from_upper[i][0][0], coordinates_lower_joint_of_gas_spring_from_upper[i][1][0])
  # Матрицы поворота СК (чтобы пружина лежала в плоскости ZX)
  matrix_rotate_cs_upper_joint_of_gas_spring[i] = rotation_matrix(0, 0, -angles_between_spring_and_ZY_of_upper_joint[i])
  # Создадим общую матрицу перемещения с последующим поворотом
  matrix_move_to_new_cs_of_upper_joint_of_gas_spring[i] = matrix_move_upper_plate[i+6].dot(matrix_rotate_cs_upper_joint_of_gas_spring[i]) 
  # Узнаем координаты нижнего шарнира точки пересечения в новой СК
  coordinates_lower_joint_gas_spring_from_upper_in_new_planes[i] = linalg.inv(matrix_move_to_new_cs_of_upper_joint_of_gas_spring[i]).dot(lower_points[i+6])
  angle_upper_joint_of_gas_spring[i] = atan2(coordinates_lower_joint_gas_spring_from_upper_in_new_planes[i][2][0], coordinates_lower_joint_gas_spring_from_upper_in_new_planes[i][1][0])

  for i in range(3):
    for j in range(len(global_coordinate_system)): 
      coordinate_system_cs_upper_joint_of_gas_spring[i][0][j] = matrix_move_to_new_cs_of_upper_joint_of_gas_spring[i].dot(matrix_1_4(global_coordinate_system[j][0],
                                                            global_coordinate_system[j][1], global_coordinate_system[j][2]))   
print(angle_upper_joint_of_gas_spring[0]*180/pi)

-5.019853642019552


# 4. Отрисовка

### Ноги

In [22]:
def grapg_leg(lower_points, global_cross_coordinates, upper_points, i, group, color):

    fig.add_trace(go.Scatter3d(x=[a_f_n(lower_points, 0)[i], a_f_n(global_cross_coordinates, 0)[i]], 
                        y=[a_f_n(lower_points, 1)[i], a_f_n(global_cross_coordinates, 1)[i]], 
                        z=[a_f_n(lower_points, 2)[i], a_f_n(global_cross_coordinates, 2)[i]],  
                        showlegend=True, name = f'Рычаг ({round(degrees_cross_matrix[i]*180/pi, 2)}) - Тяга {i+1} ({round(angle_joint_lower[i]*180/pi, 2)}; {round(angle_joint_upper[i]*180/pi, 2)})',
                        opacity=1, legendgroup=group, marker=dict(size=5, color=color, colorscale='Viridis'), line = dict(width = 2)))
    fig.add_trace(go.Scatter3d(x=[a_f_n(global_cross_coordinates, 0)[i], a_f_n(upper_points, 0)[i]], 
                        y=[a_f_n(global_cross_coordinates, 1)[i], a_f_n(upper_points, 1)[i]], 
                        z=[a_f_n(global_cross_coordinates, 2)[i], a_f_n(upper_points, 2)[i]],  
                        showlegend=False, opacity=1, legendgroup=group,marker=dict(size=5, color=color, colorscale='Viridis'), line = dict(width = 2)))                    

### Система координат

In [23]:
def coordinate_system(coordinate_system, i, base = None):

  width = 4
  opacity = 1
  
  if base:
    width = 8

  fig.add_trace(go.Scatter3d(x = [a_f_n(coordinate_system, 0)[3]],
                            y = [a_f_n(coordinate_system, 1)[3]],
                            z = [a_f_n(coordinate_system, 2)[3]],
                            showlegend=False, opacity=1, legendgroup=f"group{i}", marker=dict(size=width-1, color='gray', colorscale='Viridis',)))

  fig.add_trace(go.Scatter3d(x = [a_f_n(coordinate_system, 0)[3], a_f_n(coordinate_system, 0)[0]],
                            y = [a_f_n(coordinate_system, 1)[3], a_f_n(coordinate_system, 1)[0]],
                            z = [a_f_n(coordinate_system, 2)[3], a_f_n(coordinate_system, 2)[0]],
                showlegend=False, opacity=opacity, legendgroup=f"group{i}", marker=dict(size=1, color='red', colorscale='Viridis',), line = dict(width = width))) 
  fig.add_trace(go.Scatter3d(x = [a_f_n(coordinate_system, 0)[3], a_f_n(coordinate_system, 0)[1]],
                            y = [a_f_n(coordinate_system, 1)[3], a_f_n(coordinate_system, 1)[1]],
                            z = [a_f_n(coordinate_system, 2)[3], a_f_n(coordinate_system, 2)[1]],
                showlegend=False, opacity=opacity, legendgroup=f"group{i}", marker=dict(size=1, color='green', colorscale='Viridis',), line = dict(width = width))) 
  fig.add_trace(go.Scatter3d(x = [a_f_n(coordinate_system, 0)[3], a_f_n(coordinate_system, 0)[2]],
                            y = [a_f_n(coordinate_system, 1)[3], a_f_n(coordinate_system, 1)[2]],
                            z = [a_f_n(coordinate_system, 2)[3], a_f_n(coordinate_system, 2)[2]],
                showlegend=False, opacity=opacity, legendgroup=f"group{i}", marker=dict(size=1, color='blue', colorscale='Viridis',), line = dict(width = width)))                                      

### График

In [24]:
fig = go.Figure()

# Верхняя плита
fig.add_trace(go.Scatter3d(x = a_f_n(upper_points[:6], 0), y = a_f_n(upper_points[:6], 1), z = a_f_n(upper_points[:6], 2),
                showlegend=True, name = 'Верхняя плита', surfaceaxis=2,  opacity=0.3, legendgroup="group7",marker=dict(size=1, color='royalblue', colorscale='Viridis',)))
fig.add_trace(go.Scatter3d(x = a_f_n(upper_points[:6], 0), y = a_f_n(upper_points[:6], 1), z = a_f_n(upper_points[:6], 2),
                showlegend=False, surfaceaxis=-1, opacity=0.8, legendgroup="group7",marker=dict(size=1, color='black', colorscale='Viridis',)))
#_____________

# Нижняя плита
fig.add_trace(go.Scatter3d(x = a_f_n(lower_points[:6], 0), y = a_f_n(lower_points[:6], 1), z = a_f_n(lower_points[:6], 2),
                showlegend=True, name = 'Нижняя плита', surfaceaxis=2,  opacity=0.3, legendgroup="group8",marker=dict(size=1, color='red', colorscale='Viridis',)))
fig.add_trace(go.Scatter3d(x = a_f_n(lower_points[:6], 0), y = a_f_n(lower_points[:6], 1), z = a_f_n(lower_points[:6], 2),
                showlegend=False, surfaceaxis=-1, opacity=0.8, legendgroup="group8",marker=dict(size=1, color='black', colorscale='Viridis',)))

#_______________________________________________________________

color = ['black', 'purple', 'brown', 'green', 'orange', 'wheat']

for i in range(6):      # 
      grapg_leg(lower_points, global_cross_coordinates, upper_points, i, f"group{i}", color[i])
      coordinate_system(coordinate_system_lower_point[i][0], i)     # Локальные СК нижней плиты
      coordinate_system(coordinate_system_upper_point[i][0], i)     # Локальные СК верхней плиты
      # coordinate_system(coordinate_system_crosspoints[i][0], 9)       # Локальные СК точек пересечения
      coordinate_system(coordinate_system_crosspoints_after_rotate[i][0], i) # Локальные СК точек пересечения после поворота
coordinate_system(coordinate_system_upper_plate, 7, 1)      # локальная СК верхней плиты

for i in range(3):
      coordinate_system(coordinate_system_cs_lower_joint_of_gas_spring[i][0], i+10)
      coordinate_system(coordinate_system_cs_upper_joint_of_gas_spring[i][0], i+10)
      fig.add_trace(go.Scatter3d(x=[a_f_n(lower_points, 0)[i+6], a_f_n(upper_points, 0)[i+6]], 
                        y=[a_f_n(lower_points, 1)[i+6], a_f_n(upper_points, 1)[i+6]], 
                        z=[a_f_n(lower_points, 2)[i+6], a_f_n(upper_points, 2)[i+6]],  
                        showlegend=True, name = f'Пружина {i+1} ({round(angle_lower_joint_of_gas_spring[i]*180/pi, 2)}; {round(angle_upper_joint_of_gas_spring[i]*180/pi, 2)})',
                        opacity=1, legendgroup=f'group{i+10}', marker=dict(size=5, color=color, colorscale='Viridis'), line = dict(width = 5, dash='dot'))) 

fig.add_trace(go.Scatter3d(x = [0, 100],y = [0, 0],z = [0, 0],     # локальная СК нижней плиты
                showlegend=False, opacity=1, legendgroup="group8", marker=dict(size=1, color='red', colorscale='Viridis',), line = dict(width = 7)))
fig.add_trace(go.Scatter3d(x = [0, 0],y = [0, 100],z = [0, 0],
                showlegend=False, opacity=1, legendgroup="group8", marker=dict(size=1, color='green', colorscale='Viridis',), line = dict(width = 7)))
fig.add_trace(go.Scatter3d(x = [0, 0],y = [0, 0],z = [0, 100],
                showlegend=False, opacity=1, legendgroup="group8", marker=dict(size=1, color='blue', colorscale='Viridis',), line = dict(width = 7)))
fig.add_trace(go.Scatter3d(x = [0],y = [0],z = [0],
                            showlegend=False, opacity=1, legendgroup="group8", marker=dict(size=7, color='gray', colorscale='Viridis',)))

#_______________________________________________________________

#Оформление полей

fig.update_layout(title={'text': f"Δ = [{move_upper_CS[0]}, {move_upper_CS[1]}, {move_upper_CS[2]}, {move_upper_CS[3]}, {move_upper_CS[4]}, {move_upper_CS[5]}]; {date.today()}",
                  'y':0.97,'x':0.5,'xanchor': 'center','yanchor': 'top'})

                                             #Оформление
fig.update_layout(legend=dict(yanchor="top", y=0.99,xanchor="left", x=0.7))

tickf = 12
fig.update_layout(autosize=True, scene = {'camera_eye': {"x": -2, "y": 2, "z":1.65},'camera_center' : {"x": -0.3, "y": 0, "z":0},},
                  width=950, height=500, margin=dict(l=10, r=0, b=10, t=50))

fig.update_layout(scene=dict(xaxis=dict( title="X",backgroundcolor="rgb(200, 200, 230)",gridcolor="white",
                         showbackground=True,zerolinecolor="white", tickfont=dict(size=tickf)),
                  yaxis=dict(title="Y", backgroundcolor="rgb(230, 200,230)",gridcolor="white",
                        showbackground=True,tickfont=dict(size=tickf),zerolinecolor="white"),
                  zaxis=dict(title= "Z", backgroundcolor="rgb(200, 200,200)",gridcolor="white",
                         showbackground=True,tickfont=dict(size=tickf),zerolinecolor="white",)))

fig.update_layout(scene=dict(xaxis_showspikes=False, yaxis_showspikes=False),)
fig.update_scenes(camera_projection_type="orthographic")

fig.show()

In [27]:
solve(move_upper_CS[0],move_upper_CS[1],move_upper_CS[2],move_upper_CS[3],move_upper_CS[4],move_upper_CS[5])*180/pi

SyntaxError: invalid syntax (1511659143.py, line 1)

Собираем x,y,z,a,b,g и соответствующие им 6 углов

In [26]:
# fixed_step = 0.001

# for t in arange(0, 1 + fixed_step, fixed_step):
#     dx = 25 * sin(1 * t + 0) + 0
#     dy = 25 * sin(1 * t + 0) + 0
#     dz = 10 * sin(1 * t + 0) + 275
#     alfa = 5 * sin(1 * t + 0) + 0
#     beta = 5 * sin(1 * t + 0) + 0
#     gamma = 5 * sin(1 * t + 0) + 0
#     solve(dx, dy, dz, alfa, beta, gamma)*180/pi